In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark Example").getOrCreate()


24/12/03 10:27:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 10:27:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# RDD 생성 - 분산데이터객체


In [2]:

rdd = spark.sparkContext.parallelize( [1,2,3,4,5] )
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [4]:
rdd.take(5)

[1, 2, 3, 4, 5]

In [3]:
squared_rdd = rdd.map( lambda x : x * x)
squared_rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

In [5]:
squared_rdd.take(5)

[1, 4, 9, 16, 25]

In [6]:
squared_rdd.collect()

[1, 4, 9, 16, 25]

# 데이터프레임 객체

In [7]:
data = [ ("Alice", 1), ("Bob", 2), ("Charlie", 3)]
df = spark.createDataFrame(data, ["Name", "Value"])
df

DataFrame[Name: string, Value: bigint]

In [8]:
df.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [ ]:
df.filter()

# RDBMS 데이터 - 테이블

In [10]:
df.createOrReplaceTempView("people")

In [19]:
select_sql =  "SELECT * FROM people WHERE Value > 2 "

In [20]:
result_sql = spark.sql(select_sql)
result_sql.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



In [ ]:
# DataFrame 생성 -> select DF 추출

# MLlib

In [23]:
!pip install numpy

  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)


In [24]:
import numpy as np

In [25]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression  import LinearRegression



In [26]:
# DataFrame 생성
df = spark.createDataFrame([("Alice", 25), ("Bob", 30), ("Charlie", 35)], ["name", "age"])

# DataFrame 연산
df_filtered = df.filter(df.age > 28)
df_filtered.show()

+-------+---+
|   name|age|
+-------+---+
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [27]:
assembler = VectorAssembler(inputCols=['age'], outputCol='features')
vector_df = assembler.transform(df)

In [29]:
lr = LinearRegression(featuresCol='features', labelCol='age')
model = lr.fit(vector_df)

24/12/03 11:29:12 WARN Instrumentation: [f017c6d8] regParam is zero, which might cause numerical instability and overfitting.
24/12/03 11:29:12 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/03 11:29:12 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/12/03 11:29:12 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/12/03 11:29:12 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [30]:
pred = model.transform(vector_df)
pred.show()

+-------+---+--------+------------------+
|   name|age|features|        prediction|
+-------+---+--------+------------------+
|  Alice| 25|  [25.0]|25.000000000000036|
|    Bob| 30|  [30.0]|30.000000000000004|
|Charlie| 35|  [35.0]| 34.99999999999997|
+-------+---+--------+------------------+



In [31]:
spark.stop()